In [1]:
%load_ext autoreload
%autoreload 2
import joblib
import networkx as nx
import copy
import sys
sys.path.append("../")
from nxedit.xml_generator.xml_maker import graph_to_xml
from nxedit.xml_generator.utils import calc_tree_pos ,pos_to_str_pos,format_xml

In [22]:

g_dict=joblib.load("temp/gls.bin")

In [23]:
o_g=g_dict['normal_experiment0_graph_integrated_json_0_database/experiments/HA/exp150_eis_s86_pmps_chloranil_litfsi/exp150_eis.graphml']

In [24]:
def convert(o_g):
    g=copy.deepcopy(o_g)
    for node in g.nodes:
        #print(g.nodes[node])

        content_list=[]
        content_list.append(g.nodes[node]["label"])
        for k,v in list(g.nodes[node].items()):
            if k not in ["x","y","type","label","ID","note","sigma","keyword"]:
                content_list.append(f"{k}={v}")

            if k =="sigma":
                val=float(v.replace(" S/cm",""))
                content_list.append(f"ionic conductivity={val:.15f} S/cm")

                
            g.nodes[node]["node_name"]="\n".join(content_list)

    g = nx.relabel_nodes(g, {k:f"TASK_{k}" for k in g.nodes}, copy=False)

    return g

In [5]:
g=convert(o_g)

In [6]:
pos=calc_tree_pos(g,args='-Grankdir="LR" -Gnodesep=1 -Nshape="box" -Granksep=2 -Gsplines="ortho" ',X_SCALE=1,y_ratio=1.4,height=200)
graph_to_xml(g,pos_to_str_pos(pos),"temp/example.xml",width=150,height=140)

In [26]:
from tqdm import tqdm

split_code="vgresiofealkjgequehfdnfdhhehe"
file_path="temp/dump_gc.txt"

"""
with open(file_path, "wb") as file:
    for k in tqdm(g_dict):
        o_g=g_dict[k]
            
        g=convert(o_g)
        pos=calc_tree_pos(g,args='-Grankdir="LR" -Gnodesep=1 -Nshape="box" -Granksep=2 -Gsplines="ortho" ',X_SCALE=1,y_ratio=1.4,height=200)
        tree=graph_to_xml(g,pos_to_str_pos(pos),save_path=None,width=150,height=140)

        tree.write(file, encoding='utf-8', xml_declaration=True)
        file.write(split_code.encode())

"""

'\nwith open(file_path, "wb") as file:\n    for k in tqdm(g_dict):\n        o_g=g_dict[k]\n            \n        g=convert(o_g)\n        pos=calc_tree_pos(g,args=\'-Grankdir="LR" -Gnodesep=1 -Nshape="box" -Granksep=2 -Gsplines="ortho" \',X_SCALE=1,y_ratio=1.4,height=200)\n        tree=graph_to_xml(g,pos_to_str_pos(pos),save_path=None,width=150,height=140)\n\n        tree.write(file, encoding=\'utf-8\', xml_declaration=True)\n        file.write(split_code.encode())\n\n'

In [30]:
import pandas as pd
with open(file_path,"r") as f:
    s=f.read()

data_list=s.split(split_code)

In [32]:
df=pd.DataFrame(data_list)
df.to_excel("temp/gc.xlsx")